In [1]:
import numpy as np
import scipy as sp
from scipy import constants as co

In [2]:
eigenfreq=[5.481016689677338E9 ,7.296532954954344E9 ]
epr=[0.9603769154208086,0.010165493785847704]
lj=10*1E-9
Ej=(2.067833848E-15)**2/lj
chiepr=[[0,0],[0,0]]
for i in range(2):
    chiepr[i][i]=(epr[i]**2)*(co.hbar)*((2*np.pi*eigenfreq[i])**2)/(4*Ej)
chiepr[0][1]=np.sqrt(chiepr[0][0]*chiepr[1][1])
chiepr[1][0]=chiepr[0][1]
print("Chi matrix is", chiepr)

Chi matrix is [[67445084.06765148, 950369.5018462897], [950369.5018462897, 13391.66823683693]]


In [3]:
imyp=[1.13217E-12,6.24997E-11]
rey=[9.881713166262257E-9,1.0597430237087973E-4]
ef=[5.41E9,7.296E9]
Q=[0,0]
C=[0,0]
chibbq=[[0,0],[0,0]]
for i in range(2):
    Q[i]=ef[i]/2*imyp[i]/rey[i]
    C[i]=0.5*imyp[i]
    chibbq[i][i]=(co.e)**2/(2*lj*(C[i]**2)*(ef[i]**2)*co.h)
chibbq[0][1]=np.sqrt(chibbq[0][0]*chibbq[1][1])
chibbq[1][0]=chibbq[0][1]
zpfbbq=[0,0]
for i in range(2):
    zpfbbq[i]=np.sqrt(2/(eigenfreq[i]*imyp[i])*np.sqrt(co.hbar/2))
print("Chi matrix is",chibbq)
print("Quality factor is",Q)
print("T1 for qubit is",Q[0]/ef[0])
print(zpfbbq)

Chi matrix is [[206527048.13718176, 2774106.2301047496], [2774106.2301047496, 37262.26392774608]]
Quality factor is [309917.90577932686, 2151.4546498458567]
T1 for qubit is 5.728611936771291e-05
[4.837713617961992e-08, 5.643250109340053e-09]


In [4]:
correct_values=[[5.409,7.295],[90.1017,5141.9350],[9.6089,0.0925],[310419.73,2083.4453],[0.05738949,0.0002856],[-1.7167E02,-2.8981E-02],[-4.4611]]

In [5]:
frequencies = []
real_parts = []
imag_parts = []

with open('Impedance vs Freq 4.txt', 'r') as file:
    for line in file:
        if not line.startswith('%') and line.strip():  # Skip comments and empty lines
            parts = line.split()
            if len(parts) >= 2:
                try:
                    freq = float(parts[0])
                    impedance = complex(parts[1].replace('i', 'j'))  # Convert to Python complex format
                    frequencies.append(freq)
                    real_parts.append(impedance.real)
                    imag_parts.append(impedance.imag)
                except (ValueError, IndexError):
                    continue  

# Convert to numpy arrays for easier manipulation
freq = np.array(frequencies)
freq=freq*1E9
rey = np.array(real_parts)
imy = np.array(imag_parts)

impy=np.gradient(imy,freq)

In [6]:
#finding zeroes
zeroes,zindex,fgrad=[],[],[]
for i in range(len(imy)):
    if (imy[i-1]<0) and (imy[i]>0):
        zeroes.append(freq[i])
        zindex.append(i)
        fgrad.append((imy[i]-imy[i-1])/(freq[i]-freq[i-1]))
c,l,q,t1=[],[],[],[]
chibbq=[[0,0],[0,0]]
for i in range(len(zeroes)):
    c.append(0.5*impy[zindex[i]]/(2*np.pi))
    l.append(1/((2*np.pi*zeroes[i])**2*c[i]))
    q.append(zeroes[i]/2*impy[zindex[i]]/rey[zindex[i]])
    t1.append(q[i]/(zeroes[i]))
    chibbq[i][i]=-l[i]/lj*(co.e**2)/2*(1/(c[i]*co.h))
chibbq[0][1]=-2*np.sqrt(chibbq[0][0]*chibbq[1][1])
chibbq[1][0]=chibbq[0][1]
zeroes=np.array(zeroes)
c=np.array(c)
l=np.array(l)
q=np.array(q)
t1=np.array(t1)
chibbq=np.array(chibbq)

In [7]:
print("Resonant frequencies(GHz) =",zeroes/1E9)
print("Correct RF value=",correct_values[0])
print("--------------------------")
print("Capacitances(fF)=",c/1E-15)
print("Correct Capacitances(fF)=",correct_values[1])
print("--------------------------")
print("Inductances(nH)=",l/1E-9)
print("Correct Inductances(nH)=",correct_values[2])
print("--------------------------")
print("Quality factor=",q)
print("Correct Quality factor=",correct_values[3])
print("--------------------------")
print("T1 value=",t1/1E-3)
print("Correct T1 value=",correct_values[4])
print("--------------------------")
print("Self Kerr values=",chibbq[0][0]/1E6,chibbq[1][1]/1E6)
print("Correct Self Kerr values=",correct_values[5])
print("--------------------------")
print("Cross Kerr values =",chibbq[0][1]/1E6)
print("Correct Cross Kerr values=",correct_values[6])

Resonant frequencies(GHz) = [5.41  7.296]
Correct RF value= [5.409, 7.295]
--------------------------
Capacitances(fF)= [  90.08863876 4820.88463697]
Correct Capacitances(fF)= [90.1017, 5141.935]
--------------------------
Inductances(nH)= [9.60673273 0.09870607]
Correct Inductances(nH)= [9.6089, 0.0925]
--------------------------
Quality factor= [309895.24653093   2085.40718925]
Correct Quality factor= [310419.73, 2083.4453]
--------------------------
T1 value= [0.05728193 0.00028583]
Correct T1 value= [0.05738949, 0.0002856]
--------------------------
Self Kerr values= -206.5572513676304 -0.039659925768652085
Correct Self Kerr values= [-171.67, -0.028981]
--------------------------
Cross Kerr values = -5.7243498342491375
Correct Cross Kerr values= [-4.4611]


In [8]:
eigenfreq=[5.481016689677338E9 ,7.296532954954344E9 ]
epr=[0.9603769154208086,0.010165493785847704]
lj=10*1E-9
Ej=(2.067833848E-15)**2/lj
chiepr=[[0,0],[0,0]]
for i in range(2):
    chiepr[i][i]=-(epr[i]**2)*(co.h)*((2*np.pi*eigenfreq[i])**2)/(8*Ej)
chiepr[0][1]=-2*np.sqrt(chiepr[0][0]*chiepr[1][1])
chiepr[1][0]=chiepr[0][1]
chiepr=np.array(chiepr)
print("Chi matrix is", chiepr/1e6)

Chi matrix is [[-2.11884981e+02 -5.97134769e+00]
 [-5.97134769e+00 -4.20711666e-02]]


In [32]:
eprchi=[]
for i in range(2):
    eprchi.append(0.026/(c[i]*lj*eigenfreq[i]**2))
print(eprchi)

[0.9606851425370103, 0.01013009486353229]
